# Understannding RNN and text preprocessing using embedding

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer

In [2]:
docs = [
    "hi there",
    "how are you?",
    "this is great",
    "i am filling not good",
    "i am fine, thank you"
]

In [8]:
token = Tokenizer(oov_token="<nothing>")
token.fit_on_texts(docs)

In [9]:
token.word_index

{'<nothing>': 1,
 'you': 2,
 'i': 3,
 'am': 4,
 'hi': 5,
 'there': 6,
 'how': 7,
 'are': 8,
 'this': 9,
 'is': 10,
 'great': 11,
 'filling': 12,
 'not': 13,
 'good': 14,
 'fine': 15,
 'thank': 16}

In [10]:
token.word_counts

OrderedDict([('hi', 1),
             ('there', 1),
             ('how', 1),
             ('are', 1),
             ('you', 2),
             ('this', 1),
             ('is', 1),
             ('great', 1),
             ('i', 2),
             ('am', 2),
             ('filling', 1),
             ('not', 1),
             ('good', 1),
             ('fine', 1),
             ('thank', 1)])

In [12]:
token.document_count

5

In [17]:
sequences = token.texts_to_sequences(docs)
sequences

[[5, 6], [7, 8, 2], [9, 10, 11], [3, 4, 12, 13, 14], [3, 4, 15, 16, 2]]

## Now lets see how we get these values

In [18]:
token.word_index

{'<nothing>': 1,
 'you': 2,
 'i': 3,
 'am': 4,
 'hi': 5,
 'there': 6,
 'how': 7,
 'are': 8,
 'this': 9,
 'is': 10,
 'great': 11,
 'filling': 12,
 'not': 13,
 'good': 14,
 'fine': 15,
 'thank': 16}

here for text "hi there", "hi" is at index 5 and "there" is at index 6 so the array for the "hi there" is [5, 6]

## Padding

In [19]:
from keras.utils import pad_sequences

In [20]:
sequences = pad_sequences(sequences, padding="post")

In [21]:
sequences

array([[ 5,  6,  0,  0,  0],
       [ 7,  8,  2,  0,  0],
       [ 9, 10, 11,  0,  0],
       [ 3,  4, 12, 13, 14],
       [ 3,  4, 15, 16,  2]])

now all are of same length

# So now lets work on real dataset

In [29]:
from keras import Sequential
from keras.utils import pad_sequences
from keras.layers import Dense, SimpleRNN, Embedding, Flatten

# imdb is preprocessed dataset
from keras.datasets import imdb

In [27]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

In [30]:
X_train.shape

(25000,)

In [32]:
# this dataset is already integer encoded
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [33]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

## Padding

In [35]:
len(X_train[0])

218

In [36]:
len(X_train[1])

189

In [39]:
X_train = pad_sequences(X_train, padding="post", maxlen=50) # take only first 50 words, for just quick training
X_test = pad_sequences(X_test, padding="post", maxlen=50)

In [40]:
X_train[0]

array([2071,   56,   26,  141,    6,  194, 7486,   18,    4,  226,   22,
         21,  134,  476,   26,  480,    5,  144,   30, 5535,   18,   51,
         36,   28,  224,   92,   25,  104,    4,  226,   65,   16,   38,
       1334,   88,   12,   16,  283,    5,   16, 4472,  113,  103,   32,
         15,   16, 5345,   19,  178,   32])

In [41]:
len(X_train[0]), len(X_test[0])

(50, 50)

In [42]:
X_train.shape

(25000, 50)

## Building model

In [52]:
model = Sequential([
    SimpleRNN(32, input_shape=(50, 1), return_sequences=False),
    Dense(1, activation="sigmoid")
])

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 32)                1088      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1121 (4.38 KB)
Trainable params: 1121 (4.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [55]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 20s 22ms/step - loss: 0.6949 - accuracy: 0.4985 - val_loss: 0.6956 - val_accuracy: 0.5040
Epoch 2/5
782/782 [==============================] - 17s 21ms/step - loss: 0.6929 - accuracy: 0.5098 - val_loss: 0.6933 - val_accuracy: 0.5092
Epoch 3/5
782/782 [==============================] - 18s 22ms/step - loss: 0.6927 - accuracy: 0.5098 - val_loss: 0.6952 - val_accuracy: 0.5027
Epoch 4/5
782/782 [==============================] - 22s 28ms/step - loss: 0.6931 - accuracy: 0.5064 - val_loss: 0.6947 - val_accuracy: 0.5008
Epoch 5/5
782/782 [==============================] - 26s 33ms/step - loss: 0.6923 - accuracy: 0.5153 - val_loss: 0.6938 - val_accuracy: 0.5084


Note: this notebook is only for undersanding the text preprocessing and the RNN model not for geting the high accuracy

## Using Embedding layer

In [57]:
from keras.datasets import imdb
from keras import Sequential
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, SimpleRNN

In [58]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

In [59]:
X_train = pad_sequences(X_train, padding="post", maxlen=50)
X_test = pad_sequences(X_test, padding="post", maxlen=50)

In [60]:
X_train.shape

(25000, 50)

In [84]:
model = Sequential([
    Embedding(100000, 2), # vocab size = 10000, ouput dim = 2
    SimpleRNN(32, return_sequences=False),
    Dense(1, activation="sigmoid")
])

In [85]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 2)           200000    
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 32)                1120      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201153 (785.75 KB)
Trainable params: 201153 (785.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [87]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 25s 29ms/step - loss: 0.6795 - accuracy: 0.5478 - val_loss: 0.5814 - val_accuracy: 0.6928
Epoch 2/5
782/782 [==============================] - 36s 46ms/step - loss: 0.4582 - accuracy: 0.7855 - val_loss: 0.4620 - val_accuracy: 0.7834
Epoch 3/5
782/782 [==============================] - 45s 57ms/step - loss: 0.2791 - accuracy: 0.8878 - val_loss: 0.4823 - val_accuracy: 0.7991
Epoch 4/5
782/782 [==============================] - 27s 35ms/step - loss: 0.1813 - accuracy: 0.9330 - val_loss: 0.6570 - val_accuracy: 0.7761
Epoch 5/5
782/782 [==============================] - 24s 31ms/step - loss: 0.1187 - accuracy: 0.9595 - val_loss: 0.7171 - val_accuracy: 0.7806
